In [1]:
import pandas as pd
import os
from datetime import datetime, timedelta
import re


In [2]:
def load_options_data(ticker, start_date, end_date, data_dir):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    print(f"Fetching data for {ticker} from {start_date} to {end_date}...")
    
    while current_date <= end:
        date_str = current_date.strftime("%Y-%m-%d")
        year = current_date.strftime("%Y")
        month = current_date.strftime("%m") 
        
        file_path = os.path.join(data_dir, year, month, f"{date_str}.csv.gz")
        
        if os.path.exists(file_path):
            print(f"Processing {file_path}...")
            try:
                # Read the CSV file
                # The file has columns: ticker,conditions,correction,exchange,price,sip_timestamp,size
                df = pd.read_csv(file_path, compression='gzip')
                
                # Filter for the specific ticker
                # The ticker column format is like 'O:A251121C00125000'
                # We look for tickers starting with 'O:NVDA'
                target_prefix = f"O:{ticker}"
                df_filtered = df[df['ticker'].str.startswith(target_prefix)]
                
                if not df_filtered.empty:
                    all_data.append(df_filtered)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        else:
            # print(f"File not found: {file_path}") # Optional: reduce noise
            pass
            
        current_date += timedelta(days=1)
    
    if all_data:
        return pd.concat(all_data, ignore_index=True)
        # return combined_df
#        combined_df = pd.concat(all_data, ignore_index=True)
        # Remove duplicates that might exist in the raw data
        # initial_len = len(combined_df)
        # combined_df = combined_df.drop_duplicates()
        # if len(combined_df) < initial_len:
        #     print(f"Removed {initial_len - len(combined_df)} duplicate rows from raw data.")
        # return combined_df
    else:
        # print("No data found for the specified criteria.")
        return pd.DataFrame()


In [3]:
# Configuration
TICKER = "NVDA"
START_DATE = "2025-11-19"
END_DATE = "2025-11-23"
DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1"

# Load and merge data
stock_options = load_options_data(TICKER, START_DATE, END_DATE, DATA_DIR)

# Display results
print(f"Total rows loaded: {len(stock_options)}")
if not stock_options.empty:
    print(stock_options.head())

Fetching data for NVDA from 2025-11-19 to 2025-11-23...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1/2025/11/2025-11-19.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1/2025/11/2025-11-21.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_options_opra/trades_v1/2025/11/2025-11-21.csv.gz...
Total rows loaded: 1895664
                  ticker  conditions  correction  exchange   price  \
0  O:NVDA251121C00010000       237.0           0       312  177.35   
1  O:NVDA251121C00010000       237.0           0       312  177.29   
2  O:NVDA251121C00010000       237.0           0       312  177.26   
3  O:NVDA251121C00010000       233.0           0       302  174.32   
4  O:NVDA251121C00010000       2

In [4]:
# show the exactly duplicated rows 
exact_duplicates = stock_options[stock_options.duplicated(keep=False)]
print(exact_duplicates)

                        ticker  conditions  correction  exchange  price  \
26       O:NVDA251121C00105000       227.0           0       300  79.97   
27       O:NVDA251121C00105000       227.0           0       300  79.97   
59       O:NVDA251121C00115000       233.0           0       302  71.14   
60       O:NVDA251121C00115000       233.0           0       302  71.14   
68       O:NVDA251121C00120000       227.0           0       302  65.91   
...                        ...         ...         ...       ...    ...   
1895087  O:NVDA280121P00180000       209.0           0       325  40.20   
1895361  O:NVDA280121P00180000       209.0           0       304  41.05   
1895362  O:NVDA280121P00180000       209.0           0       304  41.05   
1895571  O:NVDA280121P00035000       227.0           0       302   0.79   
1895572  O:NVDA280121P00035000       227.0           0       302   0.79   

               sip_timestamp  size  
26       1763575724412000000     3  
27       1763575724412000

In [5]:
# drop the duplicates based on all columns
stock_options = stock_options.drop_duplicates()
print(f"Total rows after dropping duplicates: {len(stock_options)}")

Total rows after dropping duplicates: 1802132


In [6]:

# Convert sip_timestamp to datetime
# Assuming the timestamp is in nanoseconds and converting to US/Eastern
stock_options['sip_timestamp'] = pd.to_datetime(stock_options['sip_timestamp'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')


In [7]:
pattern = r'O:?([A-Z]+)(\d{6})([CP])(\d{8})'
extracted = stock_options['ticker'].str.extract(pattern)
stock_options['ticker_symbol'] = extracted[0]
stock_options['expiration'] = pd.to_datetime(extracted[1], format='%y%m%d')
stock_options['option_type'] = extracted[2].map({'C': 'call', 'P': 'put'})
stock_options['strike'] = pd.to_numeric(extracted[3]) / 1000.0

print(stock_options[['sip_timestamp', 'ticker', 'ticker_symbol', 'expiration', 'option_type', 'strike']].head())

                     sip_timestamp                 ticker ticker_symbol  \
0 2025-11-19 10:00:15.661000-05:00  O:NVDA251121C00010000          NVDA   
1 2025-11-19 10:00:52.790000-05:00  O:NVDA251121C00010000          NVDA   
2 2025-11-19 10:02:10.537000-05:00  O:NVDA251121C00010000          NVDA   
3 2025-11-19 12:07:00.679000-05:00  O:NVDA251121C00010000          NVDA   
4 2025-11-19 14:53:03.366000-05:00  O:NVDA251121C00010000          NVDA   

  expiration option_type  strike  
0 2025-11-21        call    10.0  
1 2025-11-21        call    10.0  
2 2025-11-21        call    10.0  
3 2025-11-21        call    10.0  
4 2025-11-21        call    10.0  


In [8]:
stock_options

,ticker,conditions,correction,exchange,price,sip_timestamp,size,ticker_symbol,expiration,option_type,strike
0,O:NVDA251121C00010000,237.0,0,312,177.35,2025-11-19 10:00:15.661000-05:00,10,NVDA,2025-11-21,call,10.0
1,O:NVDA251121C00010000,237.0,0,312,177.29,2025-11-19 10:00:52.790000-05:00,10,NVDA,2025-11-21,call,10.0
2,O:NVDA251121C00010000,237.0,0,312,177.26,2025-11-19 10:02:10.537000-05:00,10,NVDA,2025-11-21,call,10.0
3,O:NVDA251121C00010000,233.0,0,302,174.32,2025-11-19 12:07:00.679000-05:00,1,NVDA,2025-11-21,call,10.0
4,O:NVDA251121C00010000,233.0,0,302,175.09,2025-11-19 14:53:03.366000-05:00,1,NVDA,2025-11-21,call,10.0
...,...,...,...,...,...,...,...,...,...,...,...
1895659,O:NVDA280121P00090000,227.0,0,312,6.75,2025-11-21 13:46:53.063000-05:00,1,NVDA,2028-01-21,put,90.0
1895660,O:NVDA280121P00090000,227.0,0,323,6.64,2025-11-21 14:10:55.393000-05:00,20,NVDA,2028-01-21,put,90.0
1895661,O:NVDA280121P00095000,209.0,0,315,8.00,2025-11-21 09:30:06.033000-05:00,2,NVDA,2028-01-21,put,95.0
1895662,O:NVDA280121P00095000,209.0,0,308,8.80,2025-11-21 10:35:47.210000-05:00,1,NVDA,2028-01-21,put,95.0


In [ ]:
def load_stock_data(ticker, start_date, end_date, data_dir):
    all_data = []
    current_date = datetime.strptime(start_date, "%Y-%m-%d")
    end = datetime.strptime(end_date, "%Y-%m-%d")
    
    print(f"Fetching stock data for {ticker} from {start_date} to {end_date}...")
    
    while current_date <= end:
        date_str = current_date.strftime("%Y-%m-%d")
        year = current_date.strftime("%Y")
        month = current_date.strftime("%m")
        
        file_path = os.path.join(data_dir, year, month, f"{date_str}.csv.gz")
        
        if os.path.exists(file_path):
            print(f"Processing {file_path}...")
            try:
                # Read the CSV file
                # Columns: ticker,volume,open,close,high,low,window_start,transactions
                df = pd.read_csv(file_path, compression='gzip')
                
                # Filter for the specific ticker
                df_filtered = df[df['ticker'] == ticker].copy()
                
                if not df_filtered.empty:
                    all_data.append(df_filtered)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")
        
        current_date += timedelta(days=1)
        
    if all_data:
        return pd.concat(all_data, ignore_index=True)
    else:
        print("No stock data found for the specified criteria.")
        return pd.DataFrame()

# Configuration for Stock Data
STOCK_DATA_DIR = "/blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1"


Fetching stock data for NVDA from 2025-11-19 to 2025-11-23...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-19.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-21.csv.gz...
Total stock rows loaded: 3
  ticker     volume     open   close     high       low  \
0   NVDA  247238094  184.790  186.52  187.855  182.8301   
1   NVDA  343500467  195.950  180.64  196.000  179.8500   
2   NVDA  346920745  181.235  178.88  184.560  172.9300   

               window_start  transactions  
0 2025-11-19 00:00:00-05:00       3847201  
1 2025-11-20 00:00:00-05:00       5517617  
2 2025-11-21 00:00:00-05:00       5306174  


In [17]:
START_DATE="2025-11-15"
# Load stock data (using same date range as options)
stock_data = load_stock_data(TICKER, START_DATE, END_DATE, STOCK_DATA_DIR)

# Convert window_start to datetime (assuming nanoseconds like options)
if not stock_data.empty:
    stock_data['window_start'] = pd.to_datetime(stock_data['window_start'], unit='ns').dt.tz_localize('UTC').dt.tz_convert('US/Eastern')

print(f"Total stock rows loaded: {len(stock_data)}")
if not stock_data.empty:
    print(stock_data.head())

Fetching stock data for NVDA from 2025-11-15 to 2025-11-23...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-17.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-18.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-19.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-20.csv.gz...
Processing /blue/yixuanli/wyanbin/ResearchData/Algo/us_stocks_sip/day_aggs_v1/2025/11/2025-11-21.csv.gz...
Total stock rows loaded: 5
  ticker     volume     open   close     high       low  \
0   NVDA  173623379  185.965  186.60  189.000  184.3200   
1   NVDA  213594749  183.375  181.36  184.800  179.6500   
2   NVDA  247238094  184.790  186.52  187.855  182.8301   
3   NVDA  343500467  195.950  180.64  196.000  179.8500   
4   NVDA  346920745  181.235  178.88  184.560  172.9300   

               window

In [18]:
#rename stock_data ticker column to ticker_symbol for merging
stock_data = stock_data.rename(columns={'ticker': 'ticker_symbol'})

In [19]:
stock_data.head()

,ticker_symbol,volume,open,close,high,low,window_start,transactions
0,NVDA,173623379,185.965,186.60,189.000,184.3200,2025-11-17 00:00:00-05:00,3083602
1,NVDA,213594749,183.375,181.36,184.800,179.6500,2025-11-18 00:00:00-05:00,3539434
2,NVDA,247238094,184.790,186.52,187.855,182.8301,2025-11-19 00:00:00-05:00,3847201
3,NVDA,343500467,195.950,180.64,196.000,179.8500,2025-11-20 00:00:00-05:00,5517617
4,NVDA,346920745,181.235,178.88,184.560,172.9300,2025-11-21 00:00:00-05:00,5306174


In [20]:
stock_options.head()

,ticker,conditions,correction,exchange,price,sip_timestamp,size,ticker_symbol,expiration,option_type,strike,trade_date
0,O:NVDA251121C00010000,237.0,0,312,177.35,2025-11-19 10:00:15.661000-05:00,10,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00
1,O:NVDA251121C00010000,237.0,0,312,177.29,2025-11-19 10:00:52.790000-05:00,10,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00
2,O:NVDA251121C00010000,237.0,0,312,177.26,2025-11-19 10:02:10.537000-05:00,10,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00
3,O:NVDA251121C00010000,233.0,0,302,174.32,2025-11-19 12:07:00.679000-05:00,1,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00
4,O:NVDA251121C00010000,233.0,0,302,175.09,2025-11-19 14:53:03.366000-05:00,1,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00


In [21]:
# Create normalized date columns for accurate daily matching
stock_options['trade_date'] = stock_options['sip_timestamp'].dt.normalize()
stock_data['stock_date'] = stock_data['window_start'].dt.normalize()

# Merge to get strictly previous trading day's data
# allow_exact_matches=False ensures that if trade_date == stock_date, it's skipped, 
# finding the most recent stock_date strictly less than trade_date.
merged_data = pd.merge_asof(
    stock_options.sort_values('trade_date'),
    stock_data.sort_values('stock_date'),
    left_on='trade_date',
    right_on='stock_date',
    by='ticker_symbol',
    direction='backward',
    allow_exact_matches=False
)

In [22]:
merged_data

,ticker,conditions,correction,exchange,price,sip_timestamp,size,ticker_symbol,expiration,option_type,strike,trade_date,volume,open,close,high,low,window_start,transactions,stock_date
0,O:NVDA251121C00010000,237.0,0,312,177.35,2025-11-19 10:00:15.661000-05:00,10,NVDA,2025-11-21,call,10.0,2025-11-19 00:00:00-05:00,213594749,183.375,181.36,184.8,179.65,2025-11-18 00:00:00-05:00,3539434,2025-11-18 00:00:00-05:00
1,O:NVDA251128P00172500,209.0,0,318,3.65,2025-11-19 09:30:09.792000-05:00,4,NVDA,2025-11-28,put,172.5,2025-11-19 00:00:00-05:00,213594749,183.375,181.36,184.8,179.65,2025-11-18 00:00:00-05:00,3539434,2025-11-18 00:00:00-05:00
2,O:NVDA251128P00172500,227.0,0,308,3.43,2025-11-19 09:30:02.029000-05:00,1,NVDA,2025-11-28,put,172.5,2025-11-19 00:00:00-05:00,213594749,183.375,181.36,184.8,179.65,2025-11-18 00:00:00-05:00,3539434,2025-11-18 00:00:00-05:00
3,O:NVDA251128P00172500,232.0,0,322,3.37,2025-11-19 09:30:01.948000-05:00,1,NVDA,2025-11-28,put,172.5,2025-11-19 00:00:00-05:00,213594749,183.375,181.36,184.8,179.65,2025-11-18 00:00:00-05:00,3539434,2025-11-18 00:00:00-05:00
4,O:NVDA251128P00172500,232.0,0,312,3.42,2025-11-19 09:30:00.926000-05:00,1,NVDA,2025-11-28,put,172.5,2025-11-19 00:00:00-05:00,213594749,183.375,181.36,184.8,179.65,2025-11-18 00:00:00-05:00,3539434,2025-11-18 00:00:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1802127,O:NVDA251121P00177500,227.0,0,323,1.22,2025-11-21 09:52:46.472000-05:00,1,NVDA,2025-11-21,put,177.5,2025-11-21 00:00:00-05:00,343500467,195.950,180.64,196.0,179.85,2025-11-20 00:00:00-05:00,5517617,2025-11-20 00:00:00-05:00
1802128,O:NVDA251121P00177500,209.0,0,320,1.21,2025-11-21 09:52:46.487000-05:00,4,NVDA,2025-11-21,put,177.5,2025-11-21 00:00:00-05:00,343500467,195.950,180.64,196.0,179.85,2025-11-20 00:00:00-05:00,5517617,2025-11-20 00:00:00-05:00
1802129,O:NVDA251121P00177500,219.0,0,322,1.21,2025-11-21 09:52:46.489000-05:00,6,NVDA,2025-11-21,put,177.5,2025-11-21 00:00:00-05:00,343500467,195.950,180.64,196.0,179.85,2025-11-20 00:00:00-05:00,5517617,2025-11-20 00:00:00-05:00
1802130,O:NVDA251121P00177500,227.0,0,312,1.22,2025-11-21 09:52:46.037000-05:00,10,NVDA,2025-11-21,put,177.5,2025-11-21 00:00:00-05:00,343500467,195.950,180.64,196.0,179.85,2025-11-20 00:00:00-05:00,5517617,2025-11-20 00:00:00-05:00


In [23]:
# generate maturity days for each option trade, ignore the timezone difference

#merged_data['trade_date'] = merged_data['trade_date'].dt.tz_localize(None)
#merged_data['expiration'] = merged_data['expiration'].dt.tz_localize(None)
merged_data['maturity_days'] = (merged_data['expiration'] - merged_data['trade_date'].dt.tz_localize(None)).dt.days

In [27]:
# Calculate dollar volume (price * size * 100)
merged_data['dollar_volume'] = merged_data['size'] * merged_data['price'] /10000 # in millions

# generate summary stat for the trades and volumes for each maturity days at aggregation 
maturity_summary = merged_data.groupby('maturity_days').agg(
    total_trades=pd.NamedAgg(column='ticker', aggfunc='count'),
    total_volume=pd.NamedAgg(column='size', aggfunc='sum'),
    total_dollar_volume=pd.NamedAgg(column='dollar_volume', aggfunc='sum'),
    average_price=pd.NamedAgg(column='price', aggfunc='mean')
).reset_index()

In [28]:
maturity_summary

,maturity_days,total_trades,total_volume,total_dollar_volume,average_price
0,0,279784,2630151,336.516532,1.719974
1,1,327181,2815898,934.515814,2.611786
2,2,255571,1714807,505.603039,3.497559
3,7,161719,1357388,387.631878,3.624325
4,8,129360,1033584,412.596942,4.294151
...,...,...,...,...,...
58,757,1704,10491,40.344515,46.599888
59,758,1243,6315,24.712890,44.640249
60,791,4605,22138,91.869883,45.112969
61,792,3917,27802,100.925495,45.978547
